In [51]:
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup

In [57]:
main()

In [52]:
def get_page(league,year):
    url = 'https://understat.com/league/' + league + '/' + year #EPL/2023'
    res = requests.get(url)
    soup = BeautifulSoup(res.content,'lxml')
    scripts = soup.find_all('script')
    strings = scripts[2].string
    ind_start = strings.index("('")+2
    ind_end = strings.index("')")
    json_data = strings[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')
    data = json.loads(json_data)
    return data

In [53]:
def scrap(data):
    xg = []
    xga = []
    npxg = []
    npxga = []
    pts = []
    xpts = []
    team_name = []
    team_id = []
    h_a = []

    for key_global in data:
        temp = data[key_global]    
        data_temp_matches = temp['history']

        for index in data_temp_matches:
            for team_key in temp:
                if team_key == 'id':
                    team_id.append(temp[team_key])
                if team_key == 'title':
                    team_name.append(temp[team_key])

        for index in range(len(data_temp_matches)):
            for key in data_temp_matches[index]:
                if key =='xG':
                    xg.append(data_temp_matches[index][key])
                if key =='xGA':
                    xga.append(data_temp_matches[index][key])
                if key =='npxG':
                    npxg.append(data_temp_matches[index][key])
                if key =='npxGA':
                    npxga.append(data_temp_matches[index][key])        
                if key =='pts':
                    pts.append(data_temp_matches[index][key])
                if key =='xpts':
                    xpts.append(data_temp_matches[index][key])
                if key =='h_a':
                    h_a.append(data_temp_matches[index][key])
    col_name = ['team_name','xg','xga','npxg','npxga','pts','xpts','h_a']
    df = pd.DataFrame([team_name,xg,xga,npxg,npxga,pts,xpts,h_a],index=col_name)
    df2 = df.T
    return df2

In [54]:
def table_format(df2):
    df2_h = df2.loc[df2['h_a'] == 'h']
    df2_a = df2.loc[df2['h_a'] == 'a']
    
    df3 = df2.drop(['h_a'],axis =1 )
    df3_h = df2_h.drop(['h_a'],axis =1 )
    df3_a = df2_a.drop(['h_a'],axis =1 )
    
    df4 = df3.groupby('team_name').mean()
    df4_h = df3_h.groupby('team_name').mean()
    df4_a = df3_a.groupby('team_name').mean()
    
    played = df3[['team_name','xg']].groupby('team_name').count()
    played = played.rename(columns={"xg":"played"})
    played_h = df3_h[['team_name','xg']].groupby('team_name').count()
    played_h = played_h.rename(columns={"xg":"played"})
    played_a = df3_a[['team_name','xg']].groupby('team_name').count()
    played_a = played_a.rename(columns={"xg":"played"})
    
    df6 = pd.merge(df4, played, on='team_name', how='outer')
    df6_h = pd.merge(df4_h, played_h, on='team_name', how='outer')
    df6_a = pd.merge(df4_a, played_a, on='team_name', how='outer')
    df6_h = df6_h.rename(columns={"xg":"h_xg","xga":"h_xga","npxg":"h_npxg","npxga":"h_npxga","pts":"h_pts","xpts":"h_xpts","played":"h_pld"})
    df6_a = df6_a.rename(columns={"xg":"a_xg","xga":"a_xga","npxg":"a_npxg","npxga":"a_npxga","pts":"a_pts","xpts":"a_xpts","played":"a_pld"})
    
    df7 = pd.merge(df6, df6_h, on='team_name', how='outer')
    df7 = pd.merge(df7, df6_a, on='team_name', how='outer')
    
    for column in df7.columns[0:20]:
        df7[column] = df7[column].apply(lambda x: round(x, 2))

    return df7

In [55]:
def get_one_league(league,year):
    data = get_page(league,year)
    df2 = scrap(data)
    df7 = table_format(df2)
    return df7

In [56]:
def main():
    df_all = get_one_league('EPL','2023')
    df_all = df_all.assign(league = 'EPL')
    all_leagues = ['La_liga','Bundesliga','Ligue_1','Serie_A']
    for league in all_leagues:
        temp = get_one_league(league,'2023')
        temp = temp.assign(league = league)
        df_all = pd.concat([df_all, temp])
        
    path =  r'C:xxx/understat_big5_2023.xlsx'
    with pd.ExcelWriter(path) as writer:
        df_all.to_excel(writer, sheet_name="AllData")
        df_all.to_excel(writer, sheet_name="ExtraSheet")